In [1]:
import numpy as np
# import seq2seq
# from seq2seq.models import SimpleSeq2Seq
import keras.backend as K
import gensim
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.utils import to_categorical
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping
import numpy as np
%env CUDA_VISIBLE_DEVICES=6
import pickle

Using TensorFlow backend.


env: CUDA_VISIBLE_DEVICES=6


In [2]:
train_data = np.load("data/train.data")
test_data = np.load("data/test.data")
print(len(train_data),"個 training 音檔")
print(len(test_data),"個 testing 音檔")
max_frame_length = np.max([len(sample) for sample in train_data])
print("max langth of wav:",max_frame_length)

45036 個 training 音檔
2000 個 testing 音檔
max langth of wav: 246


In [3]:
# load caption
with open("data/train.caption","r") as f:
    train_caption = f.readlines()
    train_caption_input = ["<BOS> "+ sent.strip() for sent in train_caption]
    train_caption_target = [sent.strip()+ " <EOS>" for sent in train_caption]
# chinese character level tokenizer
tokenizer = Tokenizer(num_words=None,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=" ",
                                   char_level=False)
tokenizer.fit_on_texts(train_caption_input + train_caption_target)
print("number of token in train caption:", len(tokenizer.word_index))
input_sequences = tokenizer.texts_to_sequences(train_caption_input)
target_sequences = tokenizer.texts_to_sequences(train_caption_target)
max_length = np.max([len(i) for i in input_sequences])
print("max length:", max_length)

# pad sequence
train_input_pad = pad_sequences(input_sequences, maxlen=max_length)
train_target_pad = pad_sequences(target_sequences, maxlen=max_length)

# build training max
input_matrix = np.zeros((len(train_input_pad),max_length,len(tokenizer.word_index)+1))
target_matrix = np.zeros((len(train_target_pad),max_length,len(tokenizer.word_index)+1))
for i,sent in enumerate(train_input_pad):
    for j, word_ind in enumerate(sent):
        input_matrix[i,j,word_ind] = 1
        
for i,sent in enumerate(train_target_pad):
    for j, word_ind in enumerate(sent):
        target_matrix[i,j,word_ind] = 1
# revert : np.argmax(matrix[1],axis=1)

train_data_pad = pad_sequences(train_data, maxlen=max_frame_length,dtype='float32')

print("encode input shape:", train_data_pad.shape)
print("decode input shape:", input_matrix.shape)

number of token in train caption: 2391
max length: 14
encode input shape: (45036, 246, 39)
decode input shape: (45036, 14, 2392)


In [4]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256
num_decoder_tokens = input_matrix.shape[2]
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 39))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["acc"])
hist = History()
check_save  = ModelCheckpoint("models/model-{epoch:05d}-{val_acc:.5f}.h5",monitor='val_acc',save_best_only=True)
early_stop = EarlyStopping(monitor="val_loss", patience=2)
model.fit([train_data_pad, input_matrix], target_matrix,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.05,
         callbacks=[check_save, hist, early_stop])

Train on 42784 samples, validate on 2252 samples
Epoch 1/100
42784/42784 [==============================] - 268s - loss: 2.5902 - acc: 0.6007 - val_loss: 2.1946 - val_acc: 0.6395
Epoch 2/100
42784/42784 [==============================] - 263s - loss: 2.0521 - acc: 0.6570 - val_loss: 1.9260 - val_acc: 0.6711
Epoch 3/100
42784/42784 [==============================] - 261s - loss: 1.8357 - acc: 0.6836 - val_loss: 1.7842 - val_acc: 0.6859
Epoch 4/100
42784/42784 [==============================] - 260s - loss: 1.7173 - acc: 0.6963 - val_loss: 1.6955 - val_acc: 0.7002
Epoch 5/100
42784/42784 [==============================] - 260s - loss: 1.6357 - acc: 0.7048 - val_loss: 1.6438 - val_acc: 0.7051
Epoch 6/100
42784/42784 [==============================] - 261s - loss: 1.5761 - acc: 0.7110 - val_loss: 1.5968 - val_acc: 0.7109
Epoch 7/100
42784/42784 [==============================] - 261s - loss: 1.5271 - acc: 0.7160 - val_loss: 1.5587 - val_acc: 0.7140
Epoch 8/100
42784/42784 [================

In [6]:
# inference (testing)
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [45]:
inv_tokenizer = {v: k for k, v in tokenizer.word_index.items()}
inv_tokenizer[0] = ""

In [16]:
tokenizer.word_index["bos"]

1

In [65]:
def decode_sequence(input_frame):
    # Encode the input as state vectors.
    max_decoder_seq_length = 20
    states_value = encoder_model.predict(input_frame[np.newaxis,:,:])

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, tokenizer.word_index["bos"]] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = inv_tokenizer[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'eos' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
    decoded_sentence = decoded_sentence.replace("eos","")
    
    return decoded_sentence


In [59]:
test_data_pad = pad_sequences(test_data, maxlen=max_frame_length,dtype='float32')

In [50]:
answer = []
for i in test_data_pad:
     answer.append(decode_sequence(i))

In [47]:
inv_tokenizer[1]

'bos'

In [51]:
answer

['我們不可能會有好好的照顧孩',
 '不用了你們家裡還有一些有空',
 '不然我們要去找她爸爸媽媽看看',
 '我們不是要跟他一起去開口嗎',
 '嬰兒我們要把錢藏起來的時候',
 '我們要去找她爸爸媽媽是一個',
 '你們就不知道我們家裡的一切',
 '你們現在說好好跟妳說這麼多',
 '我們不是要去找她爸爸媽媽對',
 '你們不可以讓我們家的感情了',
 '你們有沒有跟我說一些有一些',
 '如果我們不是你們母女的心情',
 '我們不可以讓我們知道的時候',
 '我們的是不是要跟我一個人情',
 '如果你們知道了一個人在一起',
 '不知道該怎麼會這麼做的事情',
 '不然你們要不要跟我們一起去',
 '我們不是不是跟我一個人相親',
 '我一個人在外面向人家道歉是',
 '我一定要好好照顧身體會不會',
 '我們都不知道我爸爸爸媽媽在',
 '不知道該怎麼會這麼做這麼說',
 '我也不是看到這個人都不知道',
 '我們不是要去找她爸爸的時候',
 '我不知道該怎麼會在這裡做人',
 '如果我們就是不知道該怎麼辦',
 '我們要不要跟我們一起去相親',
 '我們是有一個人在家裡的人是',
 '我們不可以讓他們當我的律師',
 '你們都不可以去外面拍賣了嗎',
 '我看看看這些有一個是男朋友',
 '我們不知道該怎麼會在這裡呢',
 '我們已經掌握到關鍵的證據了',
 '如果你們不知道我們在一起嗎',
 '如果有一個人同樣的人在身邊',
 '我們不知道該怎麼會成這種事',
 '我看你們一定會好好照顧孩子',
 '不知道該怎麼會這麼做的事情',
 '不然我們要把錢交給我們一點',
 '現在是我們的事情都是不甘心',
 '不知道我們家裡不會再給我們',
 '你們不知道我爸爸爸媽媽不是',
 '因為我們一個人在一起吃飯時',
 '不知道該怎麼會在這裡做什麼',
 '一定要為了一個人在家裡面對',
 '如果我們不是說這個好好照顧',
 '你們不可以讓我們一點都不好',
 '妳應該可以去找他們交往過去',
 '現在妳們不是要去找我們一點',
 '我們不是要跟曉萍在一起吃飯',
 '我們不是要跟她一起去相親嗎',
 '不知道該怎麼會這麼做的事情',
 '我們也不會有人在身邊找方法',
 '我們已經跟楊先生說好好照顧',
 '你們不可以跟你們一起去醫院',
 '我們不是不是跟

In [54]:
states_value = encoder_model.predict(test_data_pad[1][np.newaxis,:,:])

In [67]:
decode_sequence(test_data_pad[10])

'你們有沒有跟我說一些有一些'

In [68]:
train_caption

['真 厲 害 \n',
 '對 了 文 隆 哥 \n',
 '惜 惜 \n',
 '沒 有 我 得 不 到 的 \n',
 '現 在 你 不 誇 獎 我 \n',
 '你 還 \n',
 '你 要 把 握 這 個 機 會 \n',
 '別 在 這 裡 丟 人 現 眼 \n',
 '當 然 要 知 道 他 的 想 法 \n',
 '鐵 的 紀 律 的 崇 尚 者 對 吧 \n',
 '好 不 好 \n',
 '不 要 牽 扯 我 身 邊 的 人 \n',
 '眼 睜 睜 看 她 被 你 欺 負 \n',
 '文 進 你 趕 快 打 電 話 回 去 \n',
 '我 \n',
 '不 可 以 \n',
 '妳 看 現 在 旁 邊 都 沒 有 人 \n',
 '不 來 沒 關 係 \n',
 '順 便 把 戶 口 遷 到 監 獄 裡 \n',
 '有 什 麼 事 明 天 再 說 \n',
 '好 了 \n',
 '淑 卿 \n',
 '現 在 換 我 請 你 \n',
 '叫 你 不 要 跟 文 琪 來 往 \n',
 '不 要 說 只 是 稱 讚 幾 句 \n',
 '再 見 \n',
 '就 \n',
 '其 實 換 個 立 場 \n',
 '真 的 \n',
 '為 什 麼 去 接 曉 萍 會 這 麼 久 \n',
 '無 論 妳 有 什 麼 問 題 \n',
 '阿 勇 請 兩 位 董 事 長 坐 \n',
 '妳 明 明 就 去 了 \n',
 '正 源 \n',
 '我 \n',
 '被 火 一 燻 眼 淚 就 \n',
 '不 可 能 \n',
 '搞 死 自 己 罷 了 \n',
 '也 沒 有 丟 媽 媽 的 臉 \n',
 '如 果 不 追 她 \n',
 '我 交 代 過 文 隆 \n',
 '媽 妳 先 別 看 \n',
 '是 美 純 硬 要 留 下 孩 子 \n',
 '客 人 叫 車 \n',
 '這 個 點 心 作 得 很 好 \n',
 '我 要 喝 這 個 \n',
 '賺 那 一 點 薪 水 \n',
 '好 嗎 \n',
 '與 我 無 關 \n',
 '曉 萍 \n',
 '你 嚇 到 我 了 \n',
 '放 進 去 炒 一 炒 \n',
 '記 住 \n',
 '但 是 這 個 家 \n',
 '聽 妳 這 麼 說 \n',